<a href="https://colab.research.google.com/github/mohmiim/MLIntroduction/blob/master/session-4/Session_4_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

In [33]:
import tensorflow as tf
print(tf.__version__)

if tf.__version__.startswith('2')==False :
  !pip uninstall tensorflow
  !pip install tensorflow-gpu
  print(tf.__version__)


2.0.0


In [34]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import xception
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, Conv2D, Flatten, MaxPool2D
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [0]:
# data = > https://drive.google.com/file/d/1GEKK8oRNntFyR0ZxPdcvPut-15b7CvrW/view?usp=sharing
# small Data => https://drive.google.com/file/d/1OHGNsTfvVZvWYQ7B29SYcxrLGVdeCoQb/view?usp=sharing




In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
if not os.path.exists('MLIntroData'):
    os.makedirs('MLIntroData')

# Download Zip
myzip = drive.CreateFile({'id': '1GEKK8oRNntFyR0ZxPdcvPut-15b7CvrW'})
myzip.GetContentFile('data.zip')

# 3. Unzip
zip_ref = zipfile.ZipFile('data.zip', 'r')
zip_ref.extractall('MLIntroData/data')
zip_ref.close()

In [39]:
if os.path.exists('MLIntroData'):
    print(os.listdir("MLIntroData/data/data"))


['test', 'train', '.gitignore']


In [0]:
#default sizes
Image_Width = 100
Image_Height = 100
Image_Depth = 3
targetSize = (Image_Width,Image_Height)
targetSize_withdepth = (Image_Width,Image_Height,Image_Depth)

epochs = 300
x_train = []
y_train = []
y_labels = []

In [42]:
#define the sub folders for both training and test
training = os.path.join("MLIntroData/data/data",'train')

#now the easiest way to load data is to use the ImageDataGenerator
train_data_generator = ImageDataGenerator(preprocessing_function=xception.preprocess_input,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          zoom_range=0.2,
                                          fill_mode='nearest')


train_generator = train_data_generator.flow_from_directory(training,
                                                           batch_size=20,
                                                           target_size=targetSize,
                                                           #seed=12
                                                           shuffle=False
                                                           )

Found 229 images belonging to 5 classes.


In [43]:
y_train =  train_generator.classes
for k in train_generator.class_indices.keys():
    y_labels.append(k)
y_train = to_categorical(y_train)

print(len(y_train))

229


In [44]:
# NOW WE LOAD THE PRE_TRAINED MODEL
FEATURE_EXTRACTOR = xception.Xception(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)

#now we can extract the feature and save them for our images
features_x = FEATURE_EXTRACTOR.predict_generator(train_generator)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [45]:
model = Sequential()

#add our layers
model.add(Flatten(input_shape=features_x.shape[1:]))
model.add(Dense(128,activation=relu))
model.add(Dropout(0.2))
model.add(Dense(128,activation=relu))
model.add(Dropout(0.5))
model.add(Dense(len(y_labels),activation='softmax'))
history = model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=['accuracy'])

model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               2359424   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 645       
Total params: 2,376,581
Trainable params: 2,376,581
Non-trainable params: 0
____________________________________________

In [0]:
model.fit(features_x,y_train,epochs=epochs,shuffle=True,verbose=2)

In [47]:
test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data_generator.flow_from_directory("MLIntroData/data/data/test",
                                                         target_size=(100,100),
                                                         shuffle=False)

Found 93 images belonging to 5 classes.


In [48]:
import numpy as np
x_train = []
y_train = []
y_labels = {}
x_train = np.array(x_train)
print(y_labels)
#normalize the image data same way  as the model
x_train = xception.preprocess_input(x_train)

#now we can extract the feature and save them for our images
features_x = FEATURE_EXTRACTOR.predict_generator(test_generator)
predictions = model.predict(features_x)

{}


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [49]:
from sklearn.metrics import confusion_matrix,classification_report
row_index = predictions.argmax(axis=1)
filenames = test_generator.filenames
nb_samples = len(filenames)
y_true = test_generator.classes
target_names = test_generator.class_indices.keys()
print(target_names)
print(confusion_matrix(y_true, row_index))
print('Classification Report')
target_names = test_generator.class_indices.keys()
print(classification_report(test_generator.classes, row_index, target_names=target_names))

dict_keys(['bar_chart', 'bubble_chart', 'pie_chart', 'radar_chart', 'treemap_chart'])
[[18  0  0  0  1]
 [ 0 19  0  0  0]
 [ 0  0 18  0  0]
 [ 1  0  2 15  0]
 [ 2  0  0  0 17]]
Classification Report
               precision    recall  f1-score   support

    bar_chart       0.86      0.95      0.90        19
 bubble_chart       1.00      1.00      1.00        19
    pie_chart       0.90      1.00      0.95        18
  radar_chart       1.00      0.83      0.91        18
treemap_chart       0.94      0.89      0.92        19

     accuracy                           0.94        93
    macro avg       0.94      0.94      0.94        93
 weighted avg       0.94      0.94      0.94        93

